### 05 - DATASET REDUCTION FOR DEMO

The following code does not run if any of the dataset required to pick data from previously run thorugh the launch of other codes

#### 05.1 - Reduction of the temperature and the dew point pemperature datasets

In [8]:
import xarray as xr
import pandas as pd
from datetime import datetime
import os

# === INPUT: PERCORSO FILE ===
file_path = r"C:\Users\andre\Downloads\dew_point_temp_81_23.nc"

# === INPUT: DATE DI INIZIO E FINE (formato stringa) ===
start_date_str = "2023-04-01"
end_date_str = "2023-09-30"

# === CONVERSIONE A DATETIME ===
start_date = pd.to_datetime(start_date_str)
end_date = pd.to_datetime(end_date_str)

# === APERTURA DEL DATASET ===
ds = xr.open_dataset(file_path)

# === FILTRAGGIO PER TIME RANGE GIORNALIERO 11:00–17:00 ===
time_values = pd.to_datetime(ds.time.values)
mask_date = (time_values >= start_date) & (time_values <= end_date)
mask_hour = (time_values.hour >= 11) & (time_values.hour <= 17)
final_mask = mask_date & mask_hour

# === SELEZIONE DEL SOTTOINSIEME ===
ds_subset = ds.sel(time=ds.time[final_mask])

# === CHIUSURA DEL DATASET ORIGINALE PRIMA DEL SALVATAGGIO ===
ds.close()

# === SALVATAGGIO ===
output_filename = f"2m_dew_point_temp_{start_date_str}_{end_date_str}.nc"
save_dir = r"C:\Users\andre\Downloads"
output_path = os.path.join(save_dir, output_filename)

ds_subset.to_netcdf(output_path, mode="w")  # modalità scrittura

print(f"✅ Dataset salvato correttamente in:\n{output_path}")


✅ Dataset salvato correttamente in:
C:\Users\andre\Downloads\2m_dew_point_temp_2023-04-01_2023-09-30.nc


#### 05.2 - Reduction of the heat stress indices database

In [12]:
import pandas as pd
import os

# === Parametri ===
input_path = r"C:\Users\andre\OneDrive\Documenti\Filippo\Università\24_Corsi\Geoinformatics_project\elab\Outputs\heatstress_all_timestamps_all_years.csv"
output_path = r"C:\Users\andre\OneDrive\Documenti\Filippo\Università\24_Corsi\Geoinformatics_project\elab\Outputs\heatstress_all_timestamps_all_reduced.csv"
size_limit_mb = 90

# === Caricamento solo colonna temporale per determinare gli anni disponibili ===
df_times = pd.read_csv(input_path, usecols=['Timestamp'], parse_dates=['Timestamp'])
df_times['year'] = df_times['Timestamp'].dt.year
years = sorted(df_times['year'].unique(), reverse=True)

# === Caricamento completo del CSV una sola volta ===
df_full = pd.read_csv(input_path, parse_dates=['Timestamp'])

# === Creazione progressiva del sottoinsieme completo ===
final_subset = pd.DataFrame()

for year in years:
    year_data = df_full[df_full['Timestamp'].dt.year == year]
    
    # Unione temporanea
    temp_subset = pd.concat([year_data, final_subset])
    
    # Scriviamo temporaneamente e controlliamo dimensione
    temp_subset.to_csv(output_path, index=False)
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    
    if size_mb > size_limit_mb:
        print(f"🛑 Fermato prima di aggiungere l'anno {year} perché si supererebbe il limite.")
        break
    
    # Aggiungiamo stabilmente
    final_subset = temp_subset.copy()

# Salvataggio finale
final_subset.to_csv(output_path, index=False)

# === Info finali ===
print(f"✅ File finale salvato in: {output_path}")
print(f"📦 Dimensione finale: {os.path.getsize(output_path) / (1024*1024):.2f} MB")
print(f"📆 Anni inclusi: {sorted(final_subset['Timestamp'].dt.year.unique(), reverse=True)}")


🛑 Fermato prima di aggiungere l'anno 1985 perché si supererebbe il limite.
✅ File finale salvato in: C:\Users\andre\OneDrive\Documenti\Filippo\Università\24_Corsi\Geoinformatics_project\elab\Outputs\heatstress_all_timestamps_all_reduced.csv
📦 Dimensione finale: 88.91 MB
📆 Anni inclusi: [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986]


In [11]:
import pandas as pd

df = pd.read_csv(r"C:\Users\andre\OneDrive\Documenti\Filippo\Università\24_Corsi\Geoinformatics_project\elab\Outputs\heatstress_all_timestamps_all_years.csv", nrows=5)
print(df.columns.tolist())


['Timestamp', 'Numero Timestamp', 'Indice', 'Media (°C)', 'Mediana (°C)', '95° Perc. (°C)', '99° Perc. (°C)', 'Massimo (°C)', 'Soglia (°C)', 'Media > Soglia', 'Mediana > Soglia', '95° Perc. > Soglia', '99° Perc. > Soglia', 'Massimo > Soglia', 'Data']
